In [ ]:
import os
import numba
import random
import numpy as np
import scipy.ndimage.interpolation as sni
import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2d
%matplotlib inline

def ReLU(x):
    return np.maximum(0, x)

def D_ReLU(x):
    return np.where(x < 0, 0, 1)

def L_ReLU(x):
    return np.where(x < 0, 0.001 * x, x)

def D_L_ReLU(x):
    return np.where(x < 0, 0.001, 1)

def I_L_ReLU(x):
    return np.where(x < 0, 1000 * x, x)

def softmax(x):
    for i in range(x.shape[1]):
        x[:, i] -= np.max(x[:, i])
        x[:, i] = np.exp(x[:, i])
        x[:, i] = x[:, i] / np.sum(x[:, i])
    return x

# 交叉熵损失函数
# cross-entropy cost function
def Loss(predicted, target):
    return softmax(predicted) - target

def mkdir(path):
    folder = os.path.exists(path)
    #判断是否存在文件夹如果不存在则创建为文件夹 If the folder does not exist, create a new folder
    if not folder:                   
        os.makedirs(path)            
        
@numba.njit
def RandomNoise(x):
    x = x.reshape(-1) # flat view
    for ii in range(len(x)):
        x[ii] += random.uniform(-0.1, 0.1)

In [ ]:
# 自定义卷积函数
# Convolution function

def convolution2D(input_array, kernel, stride):
    input_row, input_column = input_array.shape #行 列
    kernel_row, kernel_column = kernel.shape
    new_row = (input_row - kernel_row) // stride + 1

    new_column = (input_column - kernel_column) // stride + 1
    new_array = np.zeros((new_row, new_column))

    for j in range(new_row):
        for k in range(new_column):
            new_array[j, k] = np.sum(input_array[j * stride : j * stride + kernel_row, \
                                                        k * stride : k * stride + kernel_column] * kernel)
    
    return new_array


# 零填充
# zero padding
def padding(input_array, n):
    if n == 0:
        return input_array
    else:
        input_array_size = input_array.shape
        output_array = np.zeros((input_array_size[0], input_array_size[1] + 2 * n, input_array_size[2] + 2 * n, input_array_size[3]))
        output_array[:, n : n + input_array_size[1], n : n + input_array_size[2], :] = input_array
        return output_array

In [ ]:
# 卷积层
# Convolution layer
class ConvLayer:
    def __init__(self, input_shape, kernel_shape, stride, zero_padding, learning_rate, \
                 residual_error_flag, first_layer_flag, lock_flag, name):
                
        # 样本数 行 列 通道数
        # sample size， row, column. channel
        self.input_shape = input_shape
        self.input_number = input_shape[0] #minibatch
        
        self.input_row = input_shape[1] + 2 * zero_padding
        self.input_column = input_shape[2] + 2 * zero_padding
        self.input_channel = input_shape[3] 
                   
        self.kernel_shape = kernel_shape 
        #行 列 输入通道数 卷积核通道数
        #row, column, input channel, kernel
        
        self.kernel_size = kernel_shape[0]
        self.kernel_channel = kernel_shape[3]
        self.stride = stride 
        self.zero_padding = zero_padding
        self.learning_rate = learning_rate * (1 / self.input_number)
        self.residual_error_flag = residual_error_flag
        self.first_layer_flag = first_layer_flag
        self.lock_flag = lock_flag
        self.name = name
        
        self.conv_out_row = (self.input_row - self.kernel_size) // self.stride + 1     
        self.conv_out_column = (self.input_column - self.kernel_size) // self.stride + 1
        #self.conv_out_array = np.zeros((self.input_number, self.conv_out_row, self.conv_out_column, self.kernel_channel))
        
        # kaiming高斯初始化
        # kaiming Gaussian initialization
        #self.kernel = np.random.normal(0.0, pow(2 / (self.kernel_size * self.kernel_size * self.kernel_channel), 0.5), \
                                           #(self.kernel_size, self.kernel_size, self.kernel_channel))
        
        # xavier初始化
        # xavier initialization
        input_number = self.input_row * self.input_column * self.input_channel
        conv_out_number = self.conv_out_row * self.conv_out_column * self.kernel_channel
        
        temp_number = pow(6 / (input_number + conv_out_number), 0.5)
        
        self.kernel = np.random.uniform(-temp_number, temp_number, kernel_shape)
        #self.kernel_error = np.zeros(kernel_shape)
        
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
    # 正向查询
    # forward propagation
    def fp(self, input_array):
        
        input_array = padding(input_array, self.zero_padding)
        
        self.conv_out_array = np.zeros((input_array.shape[0], self.conv_out_row, self.conv_out_column, self.kernel_channel))
        # 卷积操作 Convolution
        for n in range(input_array.shape[0]):
            for i in range(self.kernel_channel):
                #temp_kernel = np.dstack([self.kernel[:, :, i]] * self.input_channel)
                for j in range(self.conv_out_row):
                    for k in range(self.conv_out_column):
                        self.conv_out_array[n, j, k, i] = \
                        np.sum(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                           k * self.stride : k * self.stride + self.kernel_size, :] * self.kernel[:, :, :, i])
        if self.residual_error_flag:
            return self.activation_function(self.conv_out_array) + self.conv_out_array
                    
        return self.activation_function(self.conv_out_array)
    
    
    
    # 反向查询
    # back propagation
    def bp(self, input_array, error):
        
        input_array = padding(input_array, self.zero_padding)
        
        if self.residual_error_flag:
            conv_out_error = error * (self.D_activation_function(self.conv_out_array) + 1)
        else:
            conv_out_error = error * self.D_activation_function(self.conv_out_array)

        new_array_row = (self.conv_out_row - 1) * self.stride + 1
        new_array_column = (self.conv_out_column - 1) * self.stride + 1
            
        if self.stride != 1:     
            
            
            new_array = np.zeros((self.input_number, new_array_row, new_array_column, self.kernel_channel))
            for n in range(self.input_number):
                for i in range(self.conv_out_row):
                    for j in range(self.conv_out_column):
                        new_array[n, i * self.stride , j * self.stride , :] = conv_out_error[n, i, j, :]
        else:
            new_array = conv_out_error
        
        
        
        if self.first_layer_flag :
            conv_error = None
        else:
            new_array_padding = padding(new_array, self.kernel_size - 1)
            conv_error = np.zeros(input_array.shape)
            for n in range(self.input_number):
                for i in range(self.kernel_channel):
                    for j in range(self.input_channel):
                        conv_error[n, :, :, j] += conv2d(new_array_padding[n, :, :, i], self.kernel[:, :, j, i], mode='valid')
            
            conv_error = conv_error[:, self.zero_padding : self.input_row - self.zero_padding, \
                          self.zero_padding : self.input_column - self.zero_padding, :]
            
            
        if not self.lock_flag:            
            kernel_error = np.zeros(self.kernel_shape)

            for n in range(self.input_number):
                for i in range(self.kernel_channel):
                    for j in range(self.input_channel):
                        kernel_error[:, :, j, i] += convolution2D(input_array[n, :, :, j], new_array[n, :, :, i], 1)

            self.kernel -= self.learning_rate * kernel_error     
                     
        return conv_error
    
    
    def backQuery(self, error):
        # 用于可视化反卷积 Used for visualization
        # Matthew D. Zeiler , Rob Fergus. Visualizing and Understanding Convolutional Networks.European Conference on Computer Vision, 2014 – Springer.
        
        new_array_padding = padding(error, self.kernel_size - 1)

        conv_error = np.zeros((1, self.input_row, self.input_column, self.input_channel))
        for n in range(1):
            for i in range(self.kernel_channel):
                for j in range(self.input_channel):
                    conv_error[n, :, :, j] += conv2d(new_array_padding[n, :, :, i], self.kernel[:, :, j, i], mode='valid')

        conv_error = conv_error[:, self.zero_padding : self.input_row - self.zero_padding, \
                        self.zero_padding : self.input_column - self.zero_padding, :]
        
            
            
            
        return self.activation_function(conv_error)

    
    
    def saveKernel(self, path):
        path += self.name
        mkdir(path)
        for i in range(self.kernel_channel):
            for j in range(self.input_channel):
                filename = path + '\\kernel_' + str(i) + str(j) + '.txt'
                np.savetxt(filename, self.kernel[:, :, j, i], fmt='%2.54f', delimiter=",")
        
    def loadKernel(self, path):
        path += self.name
        for i in range(self.kernel_channel):
            for j in range(self.input_channel):
                filename = path + '\\kernel_' + str(i) + str(j) + '.txt'
                self.kernel[:, :, j, i] = np.loadtxt(filename, dtype=np.float64, delimiter=',')

In [ ]:
# 池化层
class PoolingLayer:

    def __init__(self, input_shape, kernel_size, stride, mode):
        
        self.input_number, self.input_row, self.input_column, self.input_channel = input_shape #行 列 通道数
        self.kernel_size = kernel_size
        self.stride = stride
        self.mode = mode
               
        self.pooling_out_row = (self.input_row - self.kernel_size) // self.stride + 1
        self.pooling_out_column = (self.input_column - self.kernel_size) // self.stride + 1
        #self.pooling_out_array = np.zeros((self.input_number, self.pooling_out_row, self.pooling_out_column, self.input_channel))
        
        self.pooling_error = np.zeros(input_shape)
        
        self.avgpooling_parameter = 1 / (kernel_size * kernel_size)
        
        # 位置矩阵， 保存最大池化的位置信息 Save location information of max pooling
        self.location_array = np.zeros((self.input_number, self.pooling_out_row, self.pooling_out_column, \
                                        self.input_channel * 2)).astype(np.int32)
        
    # 正向查询
    def fp(self, input_array):
        
        self.pooling_out_array = np.zeros((input_array.shape[0], self.pooling_out_row, self.pooling_out_column, self.input_channel))
        # 池化操作
        if self.mode == 'max':
            for n in range(input_array.shape[0]):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                            temp_location = \
                            np.argmax(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                  k * self.stride : k * self.stride + self.kernel_size, i])
                            self.location_array[n, j, k, 2 * i] = j * self.stride + temp_location // self.kernel_size
                            self.location_array[n, j, k, 2 * i + 1] = k * self.stride + temp_location % self.kernel_size
                            self.pooling_out_array[n, j, k, i] = input_array[n, self.location_array[n, j, k, 2 * i], \
                                                                             self.location_array[n, j, k, 2 * i + 1], i]
        elif self.mode == 'avg':
            for n in range(input_array.shape[0]):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                            self.pooling_out_array[n, j, k, i] = self.avgpooling_parameter * \
                                        np.sum(input_array[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                           k * self.stride : k * self.stride + self.kernel_size, i])

        else:
            raise Exception('mode Error')  
            
            
            
        return self.pooling_out_array
    
    def bp(self, error):
        
        if self.mode == 'max':
            for n in range(self.input_number):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, self.location_array[n, j, k, 2 * i], self.location_array[n, j, k, 2 * i + 1], i] = \
                                error[n, j, k, i]

        elif self.mode == 'avg':
            for n in range(self.input_number):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, j * self.stride : j * self.stride + self.kernel_size, \
                                                k * self.stride : k * self.stride + self.kernel_size, i] += \
                                                self.avgpooling_parameter * error[n, j, k, i]
        else:
            raise Exception('mode Error') 
            
        return self.pooling_error
    def backQuery(self, error):
        if self.mode == 'max':
            for n in range(1):
                for i in range(self.input_channel):
                    for j in range(self.pooling_out_row):
                        for k in range(self.pooling_out_column):
                             self.pooling_error[n, self.location_array[n, j, k, 2 * i], self.location_array[n, j, k, 2 * i + 1], i] = \
                                error[n, j, k, i]
        return self.pooling_error

In [ ]:
# 全连接层
class FullyConnectedLayer:
    def __init__(self, input_nodes, output_nodes, learning_rate, mini_batch, residual_error_flag, lock_flag, name):
        
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate * (1 / mini_batch)
        self.mini_batch = mini_batch
        self.residual_error_flag = residual_error_flag
        self.lock_flag = lock_flag
        self.name = name
        
        #self.bias = np.ones((1, 1))
        
        # 高斯初始化权重
        #self.weight = np.random.normal(0.0, pow(2 / input_nodes, 0.5), (output_nodes, input_nodes))
        
        # xavier初始化
        temp_number = pow(6 / (self.input_nodes + self.output_nodes), 0.5)
        self.weight = np.random.uniform(-temp_number, temp_number, (self.output_nodes, self.input_nodes))
        
        # 激活函数
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
        
        
        
    def fp(self, input_array):
         
        self.output_array = np.dot(self.weight, input_array)
        
        if self.residual_error_flag:
            return self.activation_function(self.output_array) + self.output_array
        
        return self.activation_function(self.output_array)
    
    def bp(self, input_array, error):
        
        if self.residual_error_flag:
            output_error = error * (self.D_activation_function(self.output_array) + 1)
        else:
            output_error = error * self.D_activation_function(self.output_array)
        input_error = np.dot(self.weight.T, output_error)
        
        if not self.lock_flag:
            self.weight -= self.learning_rate * np.dot(output_error, np.transpose(input_array))
        
        return input_error
    
    def saveWeight(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\weight.txt'
        np.savetxt(filename, self.weight, fmt='%2.54f', delimiter=",")
        
    def loadWeight(self, path):
        path += self.name
        filename = path + '\\weight.txt'
        self.weight = np.loadtxt(filename, dtype=np.float64, delimiter=',')     
    
    
    
        
        
        

In [ ]:
# 全连接层
# Same thing as FullyConnectedLayer
class LastConnectedLayer:
    def __init__(self, input_nodes, output_nodes, learning_rate, mini_batch, residual_error_flag, lock_flag, name):
        
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate * (1 / mini_batch)
        self.mini_batch = mini_batch
        self.residual_error_flag = residual_error_flag
        self.lock_flag = lock_flag
        self.name = name
        
        #self.bias = np.ones((1, 1))
        
        # 高斯初始化权重
        #self.weight = np.random.normal(0.0, pow(2 / input_nodes, 0.5), (output_nodes, input_nodes))
        
        # xavier初始化
        temp_number = pow(6 / (self.input_nodes + self.output_nodes), 0.5)
        self.weight = np.random.uniform(-temp_number, temp_number, (self.output_nodes, self.input_nodes))
        
        # 激活函数
        self.activation_function = lambda x : L_ReLU(x)
        self.D_activation_function = lambda x : D_L_ReLU(x)
        
        
        
        
    def fp(self, input_array):
         
        self.output_array = np.dot(self.weight, input_array)
              
        return self.output_array
    
    def bp(self, input_array, error):
        
        input_error = np.dot(self.weight.T, error)
        
        if not self.lock_flag:
            self.weight -= self.learning_rate * np.dot(error, np.transpose(input_array))
        
        return input_error
    
    def saveWeight(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\weight.txt'
        np.savetxt(filename, self.weight, fmt='%2.54f', delimiter=",")
        
    def loadWeight(self, path):
        path += self.name
        filename = path + '\\weight.txt'
        self.weight = np.loadtxt(filename, dtype=np.float64, delimiter=',')     
    
    
    
        
        
        

In [ ]:
# 批标准化 Batch Normalization
# 论文作者的更复杂，此程序简化了反向传播时的一些计算量
# Reduced some computation of back propagation
# Sergey Ioffe, Christian Szegedy. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift.International Conference on Machine Learning

class BatchNormalizationLayer2D:
    def __init__(self, input_shape, learning_rate, lock_flag, name):
        self.input_row, self.input_column = input_shape
        self.learning_rate = learning_rate * (1 / self.input_column)
        self.lock_flag = lock_flag
        self.name = name
        self.mean = np.zeros((self.input_row, 1))
        self.variance = np.ones((self.input_row, 1))        
        self.beta = np.zeros((self.input_row, 1))
        self.gamma = np.ones((self.input_row, 1))
        self.momentum = 0.999

    def fp(self, input_array):
        if not self.lock_flag:
            if input_array.shape[1] != 1:
                self.mean = self.momentum * self.mean + (1 - self.momentum) * input_array.mean(axis = 1).reshape(self.input_row, 1)
                self.variance = self.momentum  * self.variance + \
                                          (1.0 - self.momentum) * input_array.var(axis = 1).reshape(self.input_row, 1)
        #Xi
        self.new_array = (input_array - self.mean) / np.sqrt(self.variance + 0.000001)
        self.output_array = self.new_array * self.gamma + self.beta
        return self.output_array
    
    def bp(self, error):
        
        new_array_error = error * self.gamma   
        if not self.lock_flag:
            self.gamma -= self.learning_rate * np.sum((error * self.new_array), axis=1).reshape(self.input_row, 1)        
            self.beta -= self.learning_rate * np.sum(error, axis=1).reshape(self.input_row, 1)
        input_error = new_array_error / np.sqrt(self.variance + 0.000001)
        
        return input_error
        
        


        
    def saveBNParameter(self, path):
        path += self.name
        mkdir(path)
        filename = path + '\\mean.txt'
        np.savetxt(filename, self.mean.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\variance.txt'
        np.savetxt(filename, self.variance.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\gamma.txt'
        np.savetxt(filename, self.gamma.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
        filename = path + '\\beta.txt'
        np.savetxt(filename, self.beta.reshape(self.input_row, 1), fmt='%2.54f', delimiter=",")
    
    def loadBNParameter(self, path):
        path += self.name
        filename = path + '\\mean.txt'
        self.mean = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\variance.txt'
        self.variance = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\gamma.txt'
        self.gamma = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        filename = path + '\\beta.txt'
        self.beta = np.loadtxt(filename, dtype=np.float64, delimiter=',').reshape(self.input_row, 1)     
        



In [ ]:
# SoftMax层
class SoftMax:
    def fp(self, input_array):
        return softmax(input_array)
    def train(self, input_array, target_array):
        return Loss(input_array, target_array)

In [ ]:
# 打开mnist数据集，加载到一个60000 * 28 * 28 * 1的矩阵中
# load the mnist data set into a matrix
training_data_file = open("D:/python/neural_network/mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

test_data_file = open("D:/python/neural_network/mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

training_data_array = np.zeros((60000, 28, 28, 1))
training_label_list = []
for i, record in enumerate(training_data_list):
    all_values = record.split(',')
    training_data_array[i, :, :, :] = np.asfarray(all_values[1:]).reshape(28, 28, 1)
    training_label_list.extend(all_values[0])
training_data_array /= 255.0
training_data_array *= 0.99
training_data_array += 0.01


del training_data_list

test_data_array = np.zeros((10000, 28, 28, 1))
test_label_list = []
for i, record in enumerate(test_data_list):
    all_values = record.split(',')
    test_data_array[i, :, :, :] = np.asfarray(all_values[1:]).reshape(28, 28, 1)
    test_label_list.extend(all_values[0])
test_data_array /= 255.0
test_data_array *= 0.99
test_data_array += 0.01

del test_data_list
    





In [ ]:
minibatch = 32
class CNN:
    def __init__(self):
        self.layer1 = ConvLayer((minibatch, 28, 28, 1), (5, 5, 1, 6), 1, 0, 0.0003, True, True, False, 'layer1')
        self.layer2 = PoolingLayer((minibatch, 24, 24, 6), 2, 2, 'max')
        self.layer3 = ConvLayer((minibatch, 12, 12, 6), (3, 3, 6, 20), 1, 0, 0.0003, True, False, False, 'layer3')
        self.layer4 = PoolingLayer((minibatch, 10, 10, 20), 2, 2, 'max')
        self.layer5 = FullyConnectedLayer(500, 120, 0.1, minibatch, False, False, 'layer5')
        self.layerx = BatchNormalizationLayer2D((120, 32), 0.1, False, 'layerx')
        self.layer6 = FullyConnectedLayer(120, 80, 0.1, minibatch, False, False, 'layer6')
        self.layer7 = LastConnectedLayer(80, 10, 0.1, minibatch, False, False, 'layer7')
        self.layer8 = SoftMax()
        
    def query(self, input_array):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        out4 = self.layer4.fp(out3).reshape(500, 1)
        out5 = self.layer5.fp(out4)
        outx = self.layerx.fp(out5)
        out6 = self.layer6.fp(outx)
        out7 = self.layer7.fp(out6)
        out8 = self.layer8.fp(out7)
        return out8
    def train(self, input_array, target_array):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        out4 = self.layer4.fp(out3).reshape(minibatch, 500).T
        out5 = self.layer5.fp(out4)
        outx = self.layerx.fp(out5)
        out6 = self.layer6.fp(outx)
        out7 = self.layer7.fp(out6)
        error1 = self.layer8.train(out7, target_array)
        error2 = self.layer7.bp(out6, error1)
        error3 = self.layer6.bp(outx, error2)
        errorx = self.layerx.bp(error3)
        error4 = self.layer5.bp(out4, errorx).T.reshape(minibatch, 5, 5, 20)
        error5 = self.layer4.bp(error4)
        error6 = self.layer3.bp(out2, error5)
        error7 = self.layer2.bp(error6)
        error8 = self.layer1.bp(input_array, error7)
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer1bp(self, input_array, n):
        out1 = self.layer1.fp(input_array)
        
        for i in range(out1.shape[3]):
            if n == i:
                continue
            else:
                out1[:, :, :, i] = 0
        xxxx = self.layer1.backQuery(out1)
        return xxxx
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer2bp(self, input_array, n):
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        
        for i in range(out2.shape[3]):
            if n != i:
 
                out2[:, :, :, i] = 0
        x1 = self.layer2.backQuery(out2)
        x2 = self.layer1.backQuery(x1)
        return x2
    
    # 用于可视化，可忽略
    # Visualization, can be ignored
    def layer3bp(self, input_array, n):
        #fig = plt.figure()
        out1 = self.layer1.fp(input_array)
        out2 = self.layer2.fp(out1)
        out3 = self.layer3.fp(out2)
        
        for i in range(out3.shape[3]):
            if n == i:
                continue
            else:
                out3[:, :, :, i] = 0

        x0 = self.layer3.backQuery(out3)
        x1 = self.layer2.backQuery(x0)
        x2 = self.layer1.backQuery(x1)
        
        return x2
    
    def saveData(self, path):
        self.layer1.saveKernel(path)
        self.layer3.saveKernel(path)
        self.layer5.saveWeight(path)
        self.layer6.saveWeight(path)
        self.layer7.saveWeight(path)
        self.layerx.saveBNParameter(path)
        
    def loadData(self, path):
        self.layer1.loadKernel(path)
        self.layer3.loadKernel(path)
        self.layer5.loadWeight(path)
        self.layer6.loadWeight(path)
        self.layer7.loadWeight(path)
        self.layerx.loadBNParameter(path)

In [ ]:
cnn = CNN()

In [ ]:
a = [-10, 0, 10] #图片旋转角度 Image rotation Angle

i = 0
j = 0
k = 0
q = 0
minibatch = 32

# 训练
input_array = np.zeros((minibatch, 28, 28, 1))
target_list = np.zeros((10, minibatch))
for n in range(60000):
    inputs = training_data_array[n, :, :, :]
    inputs = sni.rotate(inputs.reshape(28, 28), random.sample(a, 1)[0], cval=0.01, reshape=False)
    RandomNoise(inputs)
    input_array[i, :, :, 0] = inputs
    #print(np.max(inputs))
    #print(np.min(inputs))
    targets = np.zeros(10)
    targets[int(training_label_list[n])] = 1
    target_list[:, i] = targets
    i += 1
    if i == minibatch:
        cnn.train(input_array, target_list)
        i = 0
    j += 1    
    if j == 100:
        j = 0
        print(k)
        print(cnn.layer1.kernel[0, 0, 0, 0])
        #print(cnn.layery.variance[0, 0, 0, 0])
        print(cnn.layer3.kernel[0, 0, 0, 0])
        print(cnn.layer5.weight[0, 0])
        print(cnn.layer7.weight[0, 0])
        #print(cnn.layerx.variance[0, 0])
        k += 1

    





In [ ]:

scorecard = [] 
for i, correct_label in enumerate(test_label_list[0*1000:10*1000]): 
    inputs = test_data_array[i, :, :, :]
    inputs = inputs.reshape(1, 28, 28, 1) 
    outputs = cnn.query(inputs) 
    #print(outputs)
    label = np.argmax(outputs) 
    #print(label, correct_label)

    if(label == int(correct_label)): 
        scorecard.append(1) 
    else: 
        scorecard.append(0)

#print("test_T_F_list", scorecard) 
scorecard_array = np.asarray(scorecard) 
print("performance = ", scorecard_array.sum() / scorecard_array.size)

In [ ]:
a = [-10, 0, 10]
path = 'D:\\python\\neural_network\\parameter37\\count_'
counter = 0
while 1 :
    a = [-10, 0, 10]
    i = 0
    j = 0
    k = 0
    q = 0
    minibatch = 32

    # 训练
    input_array = np.zeros((minibatch, 28, 28, 1))
    target_list = np.zeros((10, minibatch))
    for n in range(60000):
        inputs = training_data_array[n, :, :, :]
        inputs = sni.rotate(inputs.reshape(28, 28), random.sample(a, 1)[0], cval=0.01, reshape=False)
        RandomNoise(inputs)
        input_array[i, :, :, 0] = inputs
        #print(np.max(inputs))
        #print(np.min(inputs))
        targets = np.zeros(10)
        targets[int(training_label_list[n])] = 1
        target_list[:, i] = targets
        i += 1
        if i == minibatch:
            cnn.train(input_array, target_list)
            i = 0

    scorecard = [] 
    for i, correct_label in enumerate(test_label_list): 
        inputs = test_data_array[i, :, :, :]
        inputs = inputs.reshape(1, 28, 28, 1) 
        outputs = cnn.query(inputs) 
        #print(outputs)
        label = np.argmax(outputs) 
        #print(label, correct_label)

        if(label == int(correct_label)): 
            scorecard.append(1) 
        else: 
            scorecard.append(0)

    #print("test_T_F_list", scorecard) 
    scorecard_array = np.asarray(scorecard) 
    counter += 1
    print(counter)
    print("performance = ", scorecard_array.sum() / scorecard_array.size)
    
    
    
    mkdir(path + str(counter))
    cnn.saveData(path + str(counter) + '\\')






    #read_mat = np.loadtxt(filename, dtype=np.float64, delimiter=',')
    #print(read_mat) 



In [ ]:
path = 'D:\\python\\neural_network\\parameter_zero\\count_'
counter = 29
cnn.loadData(path + str(counter) + '\\')